In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

# Mapping of RTC names to states
rtc_to_state = {
    "Andhra Pradesh State Road Transport Corporation": "Andhra Pradesh",
    "Assam State Transport Corporation": "Assam",
    "Bihar State Tourism Development Corporation": "Bihar",
    "Himachal Road Transport Corporation": "Himachal Pradesh",
    "Jammu and Kashmir State Road Transport Corporation": "Jammu and Kashmir",
    "Kerala RTC": "Kerala",
    "Kadamba Transport Corporation": "Kadamba",
    "Patiala and the East Punjab States Union": "Patiala",
    "Puducherry Road Transport Corporation": "Pondicherry",
    "Rajasthan State Road Transport Corporation": "Rajasthan",
    "South Bengal State Transport Corporation": "South Bengal",
    "Uttarakhand Transport Corporation": "Uttarakhand",
    "West Bengal Transport Corporation": "West Bengal",
    "North Bengal State Transport Corporation": "North Bengal",
    "Chandigarh Transport Undertaking (CTU)": "Chandigarh"
}

# Initialize the Chrome WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 15)

# Open the redBus website
driver.get("https://www.redbus.in/")
time.sleep(2)  # Allow time for the page to load
driver.maximize_window()
time.sleep(2)

# Define the XPath for RTC links
rtc_links_xpath = "//p/a"  # Target all <a> tags inside <p> tags

# Find all matching elements
rtc_elements = driver.find_elements(By.XPATH, rtc_links_xpath)

# Validate the elements and filter as needed
if len(rtc_elements) <= 3:
    print("Not enough RTC links found. Verify the XPath or webpage structure.")
    driver.quit()
    exit()

# Remove unwanted elements (excluding the first and last two)
filtered_elements = rtc_elements[1:-2]

# Initialize a list to store the dictionaries
rtc_data = []

# Extract and print the name and href link for the filtered RTC elements
for element in filtered_elements:
    try:
        rtc_name = driver.execute_script("return arguments[0].innerText;", element).strip()
        rtc_link = element.get_attribute("href")
        state_name = rtc_to_state.get(rtc_name, "Unknown")
        rtc_data.append({"RTC Name": rtc_name, "RTC Link": rtc_link, "State": state_name})
    except Exception as e:
        print(f"Error processing element: {e}")

# Close the browser
driver.quit()

# Convert the data into a DataFrame
df_rtc_data = pd.DataFrame(rtc_data)

# Save the DataFrame to a CSV file
df_rtc_data.to_csv('rtc_data.csv', index=False)

# Convert the links column to a list
rtc_routelinks_list = df_rtc_data['RTC Link'].tolist()


In [2]:
rtc_routelinks_list

['https://www.redbus.in/online-booking/apsrtc',
 'https://www.redbus.in/online-booking/astc',
 'https://www.redbus.in/online-booking/bihar-state-tourism-development-corporation',
 'https://www.redbus.in/online-booking/hrtc',
 'https://www.redbus.in/online-booking/jksrtc',
 'https://www.redbus.in/online-booking/ksrtc-kerala',
 'https://www.redbus.in/online-booking/ktcl',
 'https://www.redbus.in/online-booking/pepsu',
 'https://www.redbus.in/online-booking/puducherry-road-transport-corporation-prtc',
 'https://www.redbus.in/online-booking/rsrtc',
 'https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc',
 'https://www.redbus.in/online-booking/utc',
 'https://www.redbus.in/online-booking/west-bengal-transport-corporation',
 'https://www.redbus.in/online-booking/north-bengal-state-transport-corporation',
 'https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu']

In [3]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Initialize the Chrome WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)  # Wait up to 10 seconds for elements to load

# Load the list of main links
rtc_routelinks_list = df_rtc_data['RTC Link'].tolist()

# Initialize a list to store route names and links
all_route_data = []

# Loop through each main link
for main_link in rtc_routelinks_list:
    try:
        driver.get(main_link)  # Open the main link
        time.sleep(3)  # Wait for the page to load
        print(f"Scraping data from: {main_link}")

        # Loop through pages (assuming 5 pages; adjust logic if necessary)
        for page in range(1, 6):
            print(f"Scraping page {page} of {main_link}...")
            
            # Wait for the route elements to be present
            try:
                route_elements = wait.until(
                    EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class,'route_details')]"))
                )
            except Exception as e:
                print(f"Error finding route elements on page {page}: {e}")
                break

            # Extract route names and links
            for element in route_elements:
                try:
                    route_name = element.find_element(By.CLASS_NAME, "route").text
                    route_link = element.find_element(By.CLASS_NAME, "route").get_attribute("href")
                    all_route_data.append({
                        "main_link": main_link,
                        "route_name": route_name,
                        "route_link": route_link
                    })
                    print(f"Extracted: {route_name}, {route_link}")
                except Exception as e:
                    print(f"Error extracting route details: {e}")

            # Navigate to the next page
        try:
            page_tabs = driver.find_elements(By.XPATH, "//div[@class='DC_117_pageTabs']")  # Adjust this XPath as needed
            for tab in page_tabs:
                if tab.text == str(page + 1):  # Click the tab for the next page
                    tab.click()
                    time.sleep(2)
                    break
        except Exception as e:
            print(f"No next page found or error: {e}")
            break

    except Exception as e:
        print(f"Error processing main link {main_link}: {e}")

# Close the browser after scraping
driver.quit()

# Convert the extracted data into a DataFrame
df_route_data = pd.DataFrame(all_route_data)

# Save the DataFrame to a CSV file
df_route_data.to_csv('route_data.csv', index=False)

# Display the first few rows of the DataFrame
print("Route data successfully extracted and saved.")
print(df_route_data.head())

Scraping data from: https://www.redbus.in/online-booking/apsrtc
Scraping page 1 of https://www.redbus.in/online-booking/apsrtc...
Extracted: Hyderabad to Vijayawada, https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada
Extracted: Vijayawada to Hyderabad, https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad
Extracted: Bangalore to Tirupati, https://www.redbus.in/bus-tickets/bangalore-to-tirupathi
Extracted: Bangalore to Kadapa, https://www.redbus.in/bus-tickets/bangalore-to-kadapa
Extracted: Kakinada to Visakhapatnam, https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam
Extracted: Hyderabad to Ongole, https://www.redbus.in/bus-tickets/hyderabad-to-ongole
Extracted: Ongole to Hyderabad, https://www.redbus.in/bus-tickets/ongole-to-hyderabad
Extracted: Bangalore to Anantapur (andhra pradesh), https://www.redbus.in/bus-tickets/bangalore-to-ananthapur
Extracted: Bangalore to Chittoor (Andhra Pradesh), https://www.redbus.in/bus-tickets/bangalore-to-chittoor-andhra-pradesh
Ext

In [4]:
# Extract the list of route links from the route data
rtc_routelinks_list = df_route_data['route_link'].tolist()
print(rtc_routelinks_list)

['https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'https://www.redbus.in/bus-tickets/bangalore-to-tirupathi', 'https://www.redbus.in/bus-tickets/bangalore-to-kadapa', 'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam', 'https://www.redbus.in/bus-tickets/hyderabad-to-ongole', 'https://www.redbus.in/bus-tickets/ongole-to-hyderabad', 'https://www.redbus.in/bus-tickets/bangalore-to-ananthapur', 'https://www.redbus.in/bus-tickets/bangalore-to-chittoor-andhra-pradesh', 'https://www.redbus.in/bus-tickets/chittoor-andhra-pradesh-to-bangalore', 'https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'https://www.redbus.in/bus-tickets/bangalore-to-tirupathi', 'https://www.redbus.in/bus-tickets/bangalore-to-kadapa', 'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam', 'https://www.redbus.in/bus-tickets/hyderabad-to-ongole', 'https://www.re

In [5]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Initialize the Chrome WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Wait up to 30 seconds for elements to load

# List to store bus details
rtc_bus_details = []

def scroll():
    x = 0
    while True:
        driver.execute_script('scrollBy(0,100)')
        time.sleep(0.8)
        driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.PAGE_DOWN)
        x += 1
        if x > 175:
            break

def bus_information(route_link, route_name):
    try:
        bus_collection = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='clearfix row-one']")))

        for bus in bus_collection:
            try:
                # adding route details
                print(f"Route Name: {route_name}")
                print(f"Route Link: {route_link}")
                
                # Extract bus name
                bus_name = bus.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
                print(f"Bus Name: {bus_name}")

                # Extract bus type
                bus_type = bus.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                print(f"Bus Type: {bus_type}")

                # Extract departing time
                departing_time = bus.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
                print(f"Departing Time: {departing_time}")

                # Extract duration
                duration = bus.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
                print(f"Duration: {duration}")

                # Extract reaching time
                reaching_time = bus.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
                print(f"Reaching Time: {reaching_time}")

                # Extract star rating (with exception handling if missing)
                try:
                    star_rating = bus.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
                    print(f"Star Rating: {star_rating}")
                except Exception as e:
                    star_rating = "N/A"
                    print(f"Star Rating: {star_rating}")

                # Extract price
                price = bus.find_element(By.XPATH, ".//div[@class='seat-fare ']").text
                print(f"Price: {price}")

                # Extract seats available
                seats_available = bus.find_element(By.XPATH, ".//div[@class='column-eight w-15 fl']").text
                print(f"Seat Available: {seats_available}")

                # Append bus details to the list
                rtc_bus_details.append({
                    'Route Name': route_name,
                    'Route Link': route_link,
                    'Bus Name': bus_name,
                    'Bus Type': bus_type,
                    'Departing Time': departing_time,
                    'Duration': duration,
                    'Reaching Time': reaching_time,
                    'Star Rating': star_rating,
                    'Price': price,
                    'Seats Available': seats_available
                })

            except Exception as e:
                print(f"An error occurred while extracting bus details: {e}")
    except Exception as e:
        print(f"Error in bus_information for {route_link}: {e}")

for link in rtc_routelinks_list:
    try:
        driver.get(link)
        time.sleep(15)  # Wait for the page to load
        view_buses = driver.find_elements(By.XPATH, "//div[@class='button']")
        
        # Click on 'View' buttons to load buses
        for next_button in view_buses[::-1]:
            next_button.click()
            time.sleep(5)

        scroll()

        route_name = driver.title  # Assuming the route name is in the page title or somewhere in the page
        bus_information(link, route_name)
        
    except Exception as e:
        print(f"Error processing {link}: {e}")

# Close the browser after scraping
driver.quit()

# Convert the extracted data into a DataFrame
df_route_data = pd.DataFrame(rtc_bus_details)

# Save the DataFrame to a CSV file
df_route_data.to_csv('rtc_bus_data.csv', index=False)

# Display the first few rows of the DataFrame
print("Bus data successfully extracted and saved.")
print(df_route_data.head())

Error in bus_information for https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada: Message: 

Error in bus_information for https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad: Message: 

Error in bus_information for https://www.redbus.in/bus-tickets/bangalore-to-tirupathi: Message: 

Error in bus_information for https://www.redbus.in/bus-tickets/bangalore-to-kadapa: Message: 

Error in bus_information for https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam: Message: 

Error in bus_information for https://www.redbus.in/bus-tickets/hyderabad-to-ongole: Message: 

Error in bus_information for https://www.redbus.in/bus-tickets/ongole-to-hyderabad: Message: 

Error in bus_information for https://www.redbus.in/bus-tickets/bangalore-to-ananthapur: Message: 

Error in bus_information for https://www.redbus.in/bus-tickets/bangalore-to-chittoor-andhra-pradesh: Message: 

Error in bus_information for https://www.redbus.in/bus-tickets/chittoor-andhra-pradesh-to-bangalore: Messag

KeyboardInterrupt: 

In [ ]:
df_route_data

,main_link,route_name,route_link
0,https://www.redbus.in/online-booking/apsrtc,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,https://www.redbus.in/online-booking/apsrtc,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
2,https://www.redbus.in/online-booking/apsrtc,Bangalore to Tirupati,https://www.redbus.in/bus-tickets/bangalore-to...
3,https://www.redbus.in/online-booking/apsrtc,Bangalore to Kadapa,https://www.redbus.in/bus-tickets/bangalore-to...
4,https://www.redbus.in/online-booking/apsrtc,Kakinada to Visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
...,...,...,...
95,https://www.redbus.in/online-booking/astc,Dhubri to Guwahati,https://www.redbus.in/bus-tickets/dhubri-to-gu...
96,https://www.redbus.in/online-booking/astc,North Lakhimpur to Jorhat,https://www.redbus.in/bus-tickets/north-lakhim...
97,https://www.redbus.in/online-booking/astc,Goalpara to Guwahati,https://www.redbus.in/bus-tickets/goalpara-to-...
98,https://www.redbus.in/online-booking/astc,Sibsagar (Assam) to North Lakhimpur,https://www.redbus.in/bus-tickets/sibsagar-to-...


In [ ]:
# cleaning the column price
# this clears the column price which has Starts from\n
df_route_data["Price"] = df_route_data["Price"].str.replace("Starts from\n", "") 

# this clears the column price which has INR
df_route_data["Price"] = df_route_data["Price"].str.replace("INR", "")
df_route_data

KeyError: 'Price'

In [ ]:
# In price column some have two values the first one is the regular price and the second the one is offer price
# so we need the offer price and need to remove the regular price

# defining a function to remove the first value in price column which has two values
def correct_price(column):
    pricess=column.split()
    if len(pricess)>1:
        return pricess[1]
    return column

# applying the function
df_route_data['Price']=df_route_data['Price'].apply(correct_price)
df_route_data

KeyError: 'Price'

In [33]:
# cleaning the column Seat Available
# so hear we need only first two elements so removing the others
df_route_data["Seats Available"] = df_route_data["Seats Available"].str[:2]
df_route_data

,Route Name,Route Link,Bus Name,Bus Type,Departing Time,Duration,Reaching Time,Star Rating,Price,Seats Available
0,Hyderabad to Vijayawada Bus Online Tickets Boo...,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35188,VENNELA (A.C. SLEEPER),00:40,05h 45m,06:25,4.9,781,12
1,Hyderabad to Vijayawada Bus Online Tickets Boo...,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3563,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",01:00,06h 15m,07:15,3.5,436,31
2,Hyderabad to Vijayawada Bus Online Tickets Boo...,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35266,VENNELA (A.C. SLEEPER),01:25,05h 45m,07:10,4.1,781,3
3,Hyderabad to Vijayawada Bus Online Tickets Boo...,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3058,VENNELA (A.C. SLEEPER),02:10,05h 45m,07:55,3.9,781,24
4,Hyderabad to Vijayawada Bus Online Tickets Boo...,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3548,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",03:00,06h 20m,09:20,4.5,436,33
...,...,...,...,...,...,...,...,...,...,...
3787,Tirupati to Bangalore Bus Online Tickets Booki...,https://www.redbus.in/bus-tickets/tirupathi-to...,SVD Vedansh Travels,A/C Sleeper (2+1),05:30,02h 30m,08:00,2.3,999,25
3788,Tirupati to Bangalore Bus Online Tickets Booki...,https://www.redbus.in/bus-tickets/tirupathi-to...,Sri Tulasi Tours and Travels,A/C Sleeper (2+1),05:45,04h 15m,10:00,2.6,2300,1
3789,Tirupati to Bangalore Bus Online Tickets Booki...,https://www.redbus.in/bus-tickets/tirupathi-to...,GEETHA TRAVELS,A/C Sleeper (2+1),23:30,05h 30m,05:00,1.0,4000,32
3790,Tirupati to Bangalore Bus Online Tickets Booki...,https://www.redbus.in/bus-tickets/tirupathi-to...,KTC Travels,A/C Seater / Sleeper (2+1),23:59,06h 01m,06:00,2.2,990,31


In [34]:
# converting the data frame into a CSV file
df_route_data.to_csv('Rb_data.csv',index=False)

In [35]:
from IPython.display import FileLink
FileLink('Rb_data.csv')

C:\Users\sakhilna\Red_bus_data_akhil.csv

In [11]:
%pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [36]:
# connecting to sql
import pandas as pd
import pymysql
myconnection = pymysql.connect(host='127.0.0.1',user='root',passwd='Akhil@26')
print(myconnection)
mycursor=myconnection.cursor(pymysql.cursors.SSCursor)
# creating a seperate data base
mycursor.execute("CREATE DATABASE Red_bus")

1

In [42]:
bus_info = pd.read_csv('C:/Users/sakhilna/Desktop/Personal/Python_project/Red_bus_final_data.csv', index_col=False)
# Use the database
mycursor.execute("USE Red_bus")

# Drop the table if it already exists (optional)
mycursor.execute("DROP TABLE IF EXISTS bus_routes")

# Create the table with an `id` column
mycursor.execute("""
CREATE TABLE bus_routes (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Route_Name TEXT,
    Route_link TEXT,
    Bus_Name TEXT,
    Bus_Type TEXT,
    Departing_Time TIME,
    Duration TEXT,
    Reaching_Time TIME,
    Star_Rating FLOAT(2,1),
    Price DECIMAL(10,2),
    Seats_Available INT
)
""")

# Insert data into the table
for i, row in bus_info.iterrows():
    sql = """
    INSERT INTO bus_routes (
        Route_Name, Route_link, Bus_Name, Bus_Type, Departing_Time, Duration,
        Reaching_Time, Star_Rating, Price, Seats_Available
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    values = tuple(row)
    mycursor.execute(sql, values)
# Commit the transaction
myconnection.commit() 

In [43]:
mycursor.execute("SELECT * FROM Red_bus.bus_routes")

out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+------+---------------------------------------------------------------------------------------------+------------------------------------------------------------------------+--------------------------------------------------------+-------------------------------------------------+------------------+------------+-----------------+---------------+---------+-------------------+
|   id | Route_Name                                                                                  | Route_link                                                             | Bus_Name                                               | Bus_Type                                        | Departing_Time   | Duration   | Reaching_Time   |   Star_Rating |   Price |   Seats_Available |
|------+---------------------------------------------------------------------------------------------+------------------------------------------------------------------------+--------------------------------------------------------+----------

In [44]:
# Adding new column for AC type
mycursor.execute("USE Red_bus")
mycursor.execute("ALTER TABLE bus_routes ADD ac_type VARCHAR(10)")
mycursor.execute("SET SQL_SAFE_UPDATES = 0")
mycursor.execute("""UPDATE bus_routes
                    SET AC_type = CASE
                        WHEN LOWER(TRIM(Bus_Type)) LIKE '%non%' THEN 'Non A/C'
                        ELSE 'A/C'
                    END""")

    
        
mycursor.execute("ALTER TABLE bus_routes ADD seat_type VARCHAR(10)")
mycursor.execute("""UPDATE bus_routes
                    SET seat_type = CASE
				        WHEN LOWER(TRIM(Bus_Type)) LIKE '% sleeper / seater%' THEN 'Sleeper / Seater'
                        WHEN LOWER(TRIM(Bus_Type)) LIKE '% sleeper%' THEN 'sleeper'
                    ELSE 'Seater'
                    END""")

3792

In [45]:
myconnection.commit()

In [ ]:
########################## Scraper is complete #################################################

In [ ]:
########################## Streamlit code for website ###########################################

In [ ]:
import streamlit as st
import pandas as pd
import pymysql
from datetime import timedelta

# Database connection
def get_db_connection():
    return pymysql.connect(host='127.0.0.1', user='root', passwd='Akhil@26', database='Red_bus')

# Sidebar menu
intro = st.sidebar.radio('Main Menu', ['Home page', 'Check In for Bus Routes'])

if intro == 'Home page':
    st.title('Welcome to My Webpage 🌍')
    st.subheader('Redbus Data Scraping')
    st.write('- This involves using an automated method to extract data from the Redbus website, such as bus routes, schedules, pricing, and store it in a database for analysis.')

    # Details about the project
    st.subheader('Objective of the Project')
    st.write("""Automate the collection of bus travel data from the Redbus website using Selenium. 
             This includes gathering detailed information such as:
                    \n- Bus routes
                    \n- Ratings
                    \n- Schedules
                    \n- Ticket prices
                    \n- Seat availability""")
    st.write("Store the scraped data in a structured SQL database, making it easier to access, manage, and analyze.")
    st.write("Develop a user-friendly Streamlit application to display, filter, and analyze bus routes based on various criteria.")
    st.header("Explore Bus Routes Across India 🚀")
else:
    # Helper functions
    def fetch_distinct(column_name, condition=None):
        """Fetch distinct values from the specified column."""
        myconnection = get_db_connection()
        mycursor = myconnection.cursor()
        query = f"SELECT DISTINCT {column_name} FROM bus_routes"
        if condition:
            query += f" WHERE {condition}"
        query += f" ORDER BY {column_name}"
        mycursor.execute(query)
        result = [row[0] for row in mycursor.fetchall()]
        mycursor.close()
        myconnection.close()
        return result

    def timedelta_to_str(td):
        """Convert a timedelta object to a string (HH:MM:SS)."""
        total_seconds = int(td.total_seconds())
        hours = total_seconds // 3600
        minutes = (total_seconds % 3600) // 60
        seconds = total_seconds % 60
        return f"{hours:02}:{minutes:02}:{seconds:02}"

    # Fetch data for filters
    route_names = fetch_distinct("route_name")
    bustypes = fetch_distinct("bus_type")
    ratings = fetch_distinct("star_rating", "star_rating IS NOT NULL")
    prices = fetch_distinct("price", "price IS NOT NULL")
    ac_types = fetch_distinct("ac_type")
    seat_types = fetch_distinct("seat_type")
    departing_times = fetch_distinct("departing_time", "departing_time IS NOT NULL")

    # Parse numeric data
    min_price, max_price = float(min(prices)), float(max(prices))
    min_rating, max_rating = float(min(ratings)), float(max(ratings))

    # Time conversion for slider
    departing_time_objs = [pd.to_datetime(timedelta_to_str(t)).time() for t in departing_times]

    # Align filters properly
    st.title("🚌 Available Bus Routes")
    st.markdown("---")

    st.subheader("🔍 Filters")
    with st.container():
        col1, col2, col3 = st.columns([1, 1, 1])
        with col1:
            selected_route = st.selectbox("Select Route", route_names)
        with col2:
            selected_price = st.slider('Bus Fare Range (₹)', min_value=min_price, max_value=max_price, value=(min_price, max_price), step=500.00)
        with col3:
            selected_rating = st.slider('Rating', min_value=min_rating, max_value=max_rating, value=(min_rating, max_rating), step=0.5)

    with st.container():
        col4, col5 = st.columns(2)
        with col4:
            selected_ac = st.radio("Type", options=ac_types, horizontal=True)
        with col5:
            selected_seat = st.radio("Seat Type", options=seat_types, horizontal=True)

    col6, col7 = st.columns([2, 2])
    with col6:
        start_time, end_time = st.slider(
            'Departing Time Range',
            min_value=min(departing_time_objs),
            max_value=max(departing_time_objs),
            value=(min(departing_time_objs), max(departing_time_objs)),
            format="HH:mm:ss"
        )

    st.markdown("---")

    # Fetch filtered data
    start_time_str = start_time.strftime("%H:%M:%S")
    end_time_str = end_time.strftime("%H:%M:%S")

    query = """
        SELECT * FROM bus_routes
        WHERE route_name = %s
        AND star_rating BETWEEN %s AND %s
        AND TIME(departing_time) BETWEEN %s AND %s
        AND price BETWEEN %s AND %s
        AND ac_type = %s
        AND seat_type = %s
    """

    myconnection = get_db_connection()
    mycursor = myconnection.cursor()
    mycursor.execute(query, (selected_route, selected_rating[0], selected_rating[1],
                             start_time_str, end_time_str, selected_price[0], selected_price[1],
                             selected_ac, selected_seat))
    data = mycursor.fetchall()
    mycursor.close()
    myconnection.close()

    # Display results
    if data:
        df = pd.DataFrame(data, columns=['id', 'route_name', 'route_link', 'bus_name', 'bus_type', 
                                         'departing_time', 'duration', 'reaching_time', 'star_rating', 
                                         'price', 'seats_available', 'ac_type', 'seat_type'])
        df = df.drop(columns=['id', 'route_link'])

        st.subheader("🚌 Available Routes")
        st.dataframe(df, use_container_width=True)
        st.success('🎉 Available Routes 🎉')
    else:
        st.error("No Buses 🚍 found for the selected filters. Try different options.")